## Importing Libraries

In [0]:
import os
from google.colab import drive
import tensorflow as tf
import pandas as pd
import numpy as np
import zipfile
import nltk

In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision

tcmalloc: large alloc 1073750016 bytes == 0x5892a000 @  0x7fa9585652a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641


### Mounting Drive

In [2]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
! git clone https://github.com/facebookresearch/InferSent.git

Cloning into 'InferSent'...
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 223 (delta 0), reused 0 (delta 0), pack-reused 222
Receiving objects: 100% (223/223), 438.59 KiB | 1002.00 KiB/s, done.
Resolving deltas: 100% (112/112), done.


In [0]:
! mkdir InferSent/dataset/GloVe
! curl -Lo InferSent/dataset/GloVe/glove.840B.300d.zip http://nlp.stanford.edu/data/glove.840B.300d.zip
! unzip InferSent/dataset/GloVe/glove.840B.300d.zip -d InferSent/dataset/GloVe/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0   315    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 2075M  100 2075M    0     0  16.5M      0  0:02:05  0:02:05 --:--:-- 11.2M
Archive:  InferSent/dataset/GloVe/glove.840B.300d.zip
  inflating: InferSent/dataset/GloVe/glove.840B.300d.txt  


In [0]:
! mkdir InferSent/dataset/fastText
! curl -Lo InferSent/dataset/fastText/crawl-300d-2M.vec.zip https://s3-us-west-1.amazonaws.com/fasttext-vectors/crawl-300d-2M.vec.zip
! unzip InferSent/dataset/fastText/crawl-300d-2M.vec.zip -d InferSent/dataset/fastText/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1453M  100 1453M    0     0  21.5M      0  0:01:07  0:01:07 --:--:-- 22.1M
Archive:  InferSent/dataset/fastText/crawl-300d-2M.vec.zip
  inflating: InferSent/dataset/fastText/crawl-300d-2M.vec  


In [0]:
! curl -Lo ./InferSent/encoder/infersent1.pkl https://s3.amazonaws.com/senteval/infersent/infersent1.pkl
! curl -Lo ./InferSent/encoder/infersent2.pkl https://s3.amazonaws.com/senteval/infersent/infersent2.pkl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  146M  100  146M    0     0  21.4M      0  0:00:06  0:00:06 --:--:-- 24.6M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  146M  100  146M    0     0  23.5M      0  0:00:06  0:00:06 --:--:-- 27.4M


In [0]:
def best_results():
    grid1 = pd.read_csv("gdrive/My Drive/ASAG Work/Results/2way_tree_grid_search_new.csv",sep=',')
    grid2 = pd.read_csv("gdrive/My Drive/ASAG Work/Results/2way_tree_grid_search.csv",sep=',')    
    p = ['2_way']
    i=4
    k=6
    for c in range(6):
        p.append(max(grid1.iloc[:,i].max(),grid2.iloc[:,k].max()))
        i+=1
        k+=3
    df1 = pd.DataFrame([tuple(p)],columns=['Class']+list(grid1.columns[4:]))
    
    grid1 = pd.read_csv("gdrive/My Drive/ASAG Work/Results/3way_tree_grid_search_new.csv",sep=',')
    grid2 = pd.read_csv("gdrive/My Drive/ASAG Work/Results/3way_tree_grid_search.csv",sep=',')    
    p = ['3_way']
    i=4
    k=6
    for c in range(6):
        p.append(max(grid1.iloc[:,i].max(),grid2.iloc[:,k].max()))
        i+=1
        k+=3
    df2 = pd.DataFrame([tuple(p)],columns=['Class']+list(grid1.columns[4:]))
    
    grid1 = pd.read_csv("gdrive/My Drive/ASAG Work/Results/5way_tree_grid_search_new.csv",sep=',')
    grid2 = pd.read_csv("gdrive/My Drive/ASAG Work/Results/5way_tree_grid_search.csv",sep=',')    
    p = ['5_way']
    i=4
    k=6
    for c in range(6):
        p.append(max(grid1.iloc[:,i].max(),grid2.iloc[:,k].max()))
        i+=1
        k+=3
    df3 = pd.DataFrame([tuple(p)],columns=['Class']+list(grid1.columns[4:]))
    
    return pd.concat([df1,df2,df3]).reset_index(drop=True)

In [41]:
best_results()

,Class,UA_m_f1,UQ_m_f1,UD_m_f1,UA_w_f1,UQ_w_f1,UD_w_f1
0,2_way,0.815004,0.788492,0.770475,0.815004,0.788492,0.770475
1,3_way,0.631032,0.530458,0.472111,0.663615,0.579089,0.614325
2,5_way,0.586531,0.524664,0.406298,0.602776,0.534058,0.486732


## Loading Data

In [0]:
! cp -r 'gdrive/My Drive/ASAG Work/InferSent' .

In [0]:
from InferSent.models import InferSent
import time
import torch

V = 2
MODEL_PATH = './InferSent/encoder/infersent%s.pkl' % V
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
infersent = InferSent(params_model)
infersent.load_state_dict(torch.load(MODEL_PATH))

W2V_PATH = './InferSent/dataset/fastText/crawl-300d-2M.vec'
# W2V_PATH = './InferSent/dataset/GloVe/glove.840B.300d.txt'
infersent.set_w2v_path(W2V_PATH)

In [0]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
data = pd.read_csv("gdrive/My Drive/ASAG Work/All Data/SemEval Full Data.csv")

In [0]:
data['question'] = data['question'].astype(str)
data['reference_ans'] = data['ref_answer'].astype(str)
data['student_ans'] = data['stu_answer'].astype(str)

In [0]:
infersent.build_vocab(data['question'], tokenize=True)

Found 1387(/1406) words with w2v vectors
Vocab size : 1387


In [0]:
infersent.update_vocab(data['reference_ans'], tokenize=True)

Found 318(/322) words with w2v vectors
New vocab size : 1705 (added 318 words)


In [0]:
infersent.update_vocab(data['student_ans'], tokenize=True)

Found 2851(/3011) words with w2v vectors
New vocab size : 4556 (added 2851 words)


In [0]:
infersent = infersent.cuda()

In [0]:
def get_embeddings(data, batch_size, ltype):
    prev = time.time()
    for i in range(0,data.shape[0],batch_size):
        print(i)
        print("Time : ", time.time()-prev)
        prev = time.time()
        
        qem = np.float16(infersent.encode(data['question'].values[i:i+batch_size], tokenize=True))
        rem = np.float16(infersent.encode(data['reference_ans'].values[i:i+batch_size], tokenize=True))
        sem = np.float16(infersent.encode(data['student_ans'].values[i:i+batch_size], tokenize=True))
        
        torch.cuda.empty_cache()
        
        if(i == 0):
            q_embed = pd.DataFrame(qem)
            r_embed = pd.DataFrame(rem)
            s_embed = pd.DataFrame(sem)
        else:
            q_embed = pd.concat([q_embed,pd.DataFrame(qem)],axis=0)
            r_embed = pd.concat([r_embed,pd.DataFrame(rem)],axis=0)
            s_embed = pd.concat([s_embed,pd.DataFrame(sem)],axis=0)

        print(q_embed.shape, r_embed.shape, s_embed.shape)

    return q_embed, r_embed, s_embed

In [0]:
q_embed, r_embed, s_embed = get_embeddings(data,1000,'Train_3rd')

0
Time :  0.00028204917907714844
(1000, 4096) (1000, 4096) (1000, 4096)
1000
Time :  3.472174644470215
(2000, 4096) (2000, 4096) (2000, 4096)
2000
Time :  3.519017219543457
(3000, 4096) (3000, 4096) (3000, 4096)
3000
Time :  2.779613971710205
(4000, 4096) (4000, 4096) (4000, 4096)
4000
Time :  2.7517106533050537
(5000, 4096) (5000, 4096) (5000, 4096)
5000
Time :  6.045495510101318
(6000, 4096) (6000, 4096) (6000, 4096)
6000
Time :  6.765094041824341
(7000, 4096) (7000, 4096) (7000, 4096)
7000
Time :  6.591816663742065
(8000, 4096) (8000, 4096) (8000, 4096)
8000
Time :  7.3741700649261475
(9000, 4096) (9000, 4096) (9000, 4096)
9000
Time :  5.820080518722534
(10000, 4096) (10000, 4096) (10000, 4096)
10000
Time :  3.2437891960144043
(11000, 4096) (11000, 4096) (11000, 4096)
11000
Time :  3.59075665473938
(12000, 4096) (12000, 4096) (12000, 4096)
12000
Time :  2.824627637863159
(13000, 4096) (13000, 4096) (13000, 4096)
13000
Time :  3.3433258533477783
(14000, 4096) (14000, 4096) (14000, 40

In [0]:
q_embed.shape, r_embed.shape, s_embed.shape

((48249, 4096), (48249, 4096), (48249, 4096))

In [0]:
q_embed.to_csv("gdrive/My Drive/ASAG Work/All Data/SemEval_Question_Infersent_Embeddings.csv",sep=',',index=None)

In [0]:
r_embed.to_csv("gdrive/My Drive/ASAG Work/All Data/SemEval_Ref_Ans_Infersent_Embeddings.csv",sep=',',index=None)

In [0]:
s_embed.to_csv("gdrive/My Drive/ASAG Work/All Data/SemEval_Stu_Ans_Infersent_Embeddings.csv",sep=',',index=None)

### Functions

In [0]:
## Global Variables

questions = {}
ref_answer = {}
grades = {}
answers = {}
key={}

In [0]:
def set_key(file):
  x = 0
  count = 0
  for line in file:
    if line[0] != '#':
      key[count] = line
      x = line.index('\n')
      key[count] = line[:x]
      count = count + 1

def get_info(file, loc):
  ind = 0
  for i in range(len(file)):
    ind = file[i].find(' ')
    num = file[i][:ind]
    for j in key.keys():
      if key[j] == num:
        loc[j] = file[i][ind+1:]

In [0]:
def get_answers(file):
  print(len(file))
  prev, current = '1.0','1.1'
  temp_answers = {}
  ind, counter = 0, 0
  for i in range(len(file)):
    ind = file[i].find(' ')
    num = file[i][:ind]
    if(num not in key.values()):
      continue
    if num==current:
      temp_answers[counter] = file[i][ind+1:]
      counter = counter + 1
      prev = current
    else:
      for j in key.keys():
        if(key[j] == prev):
          answers[j] = temp_answers
          temp_answers = {}
          counter = 0
          current = num
          temp_answers[counter] = file[i][ind+1:]
          counter = counter + 1
  for j in key.keys():
    if(key[j] == prev):
      answers[j] = temp_answers

In [0]:
def get_scores(file):
  for i in key.keys():
    data = open(file+key[i]+"/ave","r").readlines()
    temp_scores = {}
    counter = 0 
    for sc in data:
      x = sc.index('\n')
      temp_scores[counter] = sc[:x]
      counter = counter + 1
    grades[i] = temp_scores

### Extracting queries, reference_answer, grades and responses

In [0]:
filename = "gdrive/My Drive/ASAG/ASAG Work/ShortAnswerGrading_v2.0/"
data = open(filename+"/data/docs/files","r").readlines()
set_key(data)
data = open(filename+"/data/sent/questions","r").readlines()
get_info(data, questions)
data = open(filename+"/data/sent/answers","r").readlines()
get_info(data, ref_answer)
data = open(filename+"/data/sent/all","r").readlines()
get_answers(data)
data = filename+"data/scores/"
get_scores(data)

2442


In [0]:
print(len(answers), len(questions), len(ref_answer), len(grades))

81 81 81 81


In [0]:
grades[30]

{0: '5',
 1: '5',
 2: '5',
 3: '5',
 4: '4.5',
 5: '4.5',
 6: '4.5',
 7: '5',
 8: '5',
 9: '5',
 10: '4.5',
 11: '5',
 12: '4.5',
 13: '5',
 14: '3.5',
 15: '5',
 16: '5',
 17: '5',
 18: '4',
 19: '5',
 20: '5',
 21: '5',
 22: '5',
 23: '4.5',
 24: '4.5',
 25: '4.5'}

['What',
 'is',
 'the',
 'role',
 'of',
 'a',
 'prototype',
 'program',
 'in',
 'problem',
 'solving?',
 'stages',
 'software',
 'life',
 'cycle',
 'are',
 'influenced',
 'by',
 'testing',
 'stage?',
 'main',
 'advantages',
 'associated',
 'with',
 'object-oriented',
 'programming?',
 'Where',
 'do',
 'C++',
 'programs',
 'begin',
 'to',
 'execute?',
 'variable?',
 'variables',
 'declared',
 'program?',
 'typically',
 'included',
 'class',
 'definition?',
 'difference',
 'between',
 'data',
 'member',
 'and',
 'local',
 'variable',
 'inside',
 'function?',
 'constructor',
 'When',
 'does',
 'create',
 'default',
 'constructor?',
 'How',
 'many',
 'constructors',
 'can',
 'be',
 'created',
 'for',
 'class?',
 'function',
 'signature',
 'include?',
 'scope',
 'global',
 'variables?',
 'compiler',
 'handle',
 'inline',
 'functions?',
 'advantage',
 'arguments',
 'that',
 'passed',
 'reference?',
 'overloaded',
 'functions',
 'differentiated',
 'compiler?',
 'defining',
 'recursive',
 'fun

## Converting to Glove

### Getting pre-trained Glove Embeddings

In [0]:
# from gensim.scripts.glove2word2vec import glove2word2vec
# glove_input_file = 'gdrive/My Drive/ASAG/Mohler/glove.6B.100d.txt'
# word2vec_output_file = 'gdrive/My Drive/ASAG/Mohler/glove.6B.100d.txt.word2vec'
# glove2word2vec(glove_input_file, word2vec_output_file)

In [0]:

# from gensim.models import KeyedVectors
# # load the Stanford GloVe model
# filename = 'gdrive/My Drive/ASAG/Mohler/glove.6B.100d.txt.word2vec'
# model = KeyedVectors.load_word2vec_format(filename, binary=False)
# # calculate: (king - man) + woman = ?
# result = model.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
# print(result)

In [0]:
# model.get_vector("man")

### Tokenzing words

In [0]:
qs=questions
ra=ref_answer
a=answers
g=grades

for q in questions.keys():
  qs[q] = word_tokenize(questions[q])
  ra[q] = word_tokenize(ref_answer[q])
for k in answers.keys():
  for response in answers[k]:
    a[k][response] = word_tokenize(answers[k][response].lower().replace("<br>",""))  

LookupError: ignored

In [0]:
qs

NameError: ignored

### Training our own Glove model

In [0]:

common_list=[]
for q in qs.keys():
  common_list.append(qs[q])
for ans in ra.keys():
  common_list.append(ra[ans])


In [0]:
for k in a.keys():
  for response in a[k]:
    common_list.append(a[k][response])

In [0]:
common_list[600]

['they',
 'are',
 'both',
 'based',
 'on',
 'a',
 'control',
 'statement',
 ',',
 'both',
 'involve',
 'repetition',
 ',',
 'both',
 'involve',
 'a',
 'termination',
 'case',
 ',',
 'both',
 'graduatlly',
 'approach',
 'that',
 'termination',
 'case',
 ',',
 'and',
 'both',
 'can',
 'occur',
 'infinitely']

In [0]:
word = []
for i in range(len(common_list)):
  for j in range(len(common_list[i])):
    if(common_list[i][j] not in word):
      word.append(common_list[i][j])

In [0]:
word

['what',
 'is',
 'the',
 'role',
 'of',
 'a',
 'prototype',
 'program',
 'in',
 'problem',
 'solving',
 '?',
 'stages',
 'software',
 'life',
 'cycle',
 'are',
 'influenced',
 'by',
 'testing',
 'stage',
 'main',
 'advantages',
 'associated',
 'with',
 'object-oriented',
 'programming',
 'where',
 'do',
 'c++',
 'programs',
 'begin',
 'to',
 'execute',
 'variable',
 'variables',
 'declared',
 'typically',
 'included',
 'class',
 'definition',
 'difference',
 'between',
 'data',
 'member',
 'and',
 'local',
 'inside',
 'function',
 'constructor',
 'when',
 'does',
 'create',
 'default',
 'how',
 'many',
 'constructors',
 'can',
 'be',
 'created',
 'for',
 'signature',
 'include',
 'scope',
 'global',
 'compiler',
 'handle',
 'inline',
 'functions',
 'advantage',
 'arguments',
 'that',
 'passed',
 'reference',
 'overloaded',
 'differentiated',
 'defining',
 'recursive',
 ',',
 'possible',
 'causes',
 'infinite',
 'recursion',
 'similarities',
 'iteration',
 'simulate',
 'behaviour',
 'po

In [0]:
len(word)

1308

In [0]:
v_len = int(np.ceil(np.sqrt(np.sqrt(len(word)))))

In [0]:
lines = common_list
#importing the glove library
from glove import Corpus, Glove

# creating a corpus object
corpus = Corpus() 

#training the corpus to generate the co occurence matrix which is used in GloVe
#window is the distance between two words algo should consider to find some relationship between them
corpus.fit(lines, window=10)

#creating a Glove object which will use the matrix created in the above lines to create embeddings
#We can set the learning rate as it uses Gradient Descent and number of components
#No_components is the dimensions of output vector
glove = Glove(no_components=v_len, learning_rate=0.05)
 
glove.fit(corpus.matrix, epochs=1500, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)
glove.save('glove.model')

Performing 1500 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79
Epoch 80
Epoch 81
Epoch 82
Epoch 83
Epoch 84
Epoch 85
Epoch 86
Epoch 87
Epoch 88
Epoch 89
Epoch 90
Epoch 91
Epoch 92
Epoch 93
Epoch 94
Epoch 95
Epoch 96
Epoch 97
Epoch 98
Epoch 99
Epoch 100
Epoch 101
Epoch 102
Epoch 103
Epoch 104
Epoch 105
Epo

In [0]:
print (glove.word_vectors[glove.dictionary[qs[0][0]]])

[-0.07941861  0.4468406   0.04821956 -0.16253219 -0.44233967  0.63009201
 -0.21562715]


In [0]:
qsv=qs
rav=ra
av=a

for q in qs.keys():
  for i in range(len(qs[q])):
    qsv[q][i] = glove.word_vectors[glove.dictionary[ qs[q][i] ]]
  for j in range(len(rav[q])):  
    rav[q][j] = glove.word_vectors[glove.dictionary[ ra[q][j] ]]
for k in a.keys():
  for response in a[k]:
    for p in range(len(a[k][response])):
      av[k][response][p] = glove.word_vectors[glove.dictionary[ a[k][response][p] ]]  


In [0]:
max_ql = 0
max_rl = 0
max_al = 0
for q in qsv.keys():
  if len(qsv[q]) > max_ql:
    max_ql = len(qsv[q])

for r in rav.keys():
  if len(rav[r]) > max_rl:
    max_rl = len(rav[r])

for a in av.keys():
  for resp in av[a]:
    if len(av[a][resp]) > max_al:
      max_al = len(av[a][resp])
print ("max Lengths  : " , max_ql, max_rl, max_al)

max Lengths  :  17 37 193


In [0]:
def padding(vect, max_len, v_len):
  diff = max_len - len(vect)
  to_append = np.zeros((diff,v_len),dtype=np.float64)
  return np.concatenate((vect,to_append),axis=0)

In [0]:
for q in qsv.keys():
  qsv[q] = padding(qsv[q], max_ql, v_len)
for r in rav.keys():
  rav[r] = padding(rav[r], max_rl, v_len)
for a in av.keys():
  for resp in av[a]:
    av[a][resp] = padding(av[a][resp], max_al, v_len)  

## Model

In [0]:
import keras.layers as L
import keras.models as M
import keras.optimizers as O
import keras.backend as K

Using TensorFlow backend.


In [0]:
q_inp = L.Input(shape=(max_ql,v_len))
r_inp = L.Input(shape=(max_rl,v_len))
a_inp = L.Input(shape=(max_al,v_len))

q_cnn = L.Conv1D(256,3)(q_inp)
q_cnn = L.MaxPool1D(2)(q_cnn)
q_cnn = L.BatchNormalization()(q_cnn)
q_cnn = L.Conv1D(128,3)(q_cnn)
q_cnn = L.MaxPool1D(2)(q_cnn)
q_cnn = L.BatchNormalization()(q_cnn)

q_rnn = L.LSTM(256,recurrent_dropout=0.2,return_sequences=True,unroll=True)(q_cnn)
# q_rnn = L.LSTM(256,recurrent_dropout=0.2,return_sequences=True,unroll=True)(q_rnn)
# q_rnn = L.LSTM(256,recurrent_dropout=0.2,return_sequences=True,unroll=True)(q_rnn)

q_avg = L.Lambda(lambda y: K.mean(y, axis=1))(q_rnn)
# q_avg = L.Lambda(lambda y: K.squeeze(y, axis=1))(q_avg)

r_cnn = L.Conv1D(256,3)(r_inp)
r_cnn = L.MaxPool1D(2)(r_cnn)
r_cnn = L.BatchNormalization()(r_cnn)
r_cnn = L.Conv1D(128,3)(r_cnn)
r_cnn = L.MaxPool1D(2)(r_cnn)
r_cnn = L.BatchNormalization()(r_cnn)

r_rnn = L.LSTM(256,recurrent_dropout=0.2,return_sequences=True,unroll=True)(r_cnn)
# r_rnn = L.LSTM(256,recurrent_dropout=0.2,return_sequences=True,unroll=True)(r_rnn)
# r_rnn = L.LSTM(256,recurrent_dropout=0.2,return_sequences=True,unroll=True)(r_rnn)

r_avg = L.Lambda(lambda y: K.mean(y, axis=1))(r_rnn)

#for answers
a_cnn = L.Conv1D(256,3)(a_inp)
a_cnn = L.MaxPool1D(2)(a_cnn)
a_cnn = L.BatchNormalization()(a_cnn)
a_cnn = L.Conv1D(128,3)(a_cnn)
a_cnn = L.MaxPool1D(2)(a_cnn)
a_cnn = L.BatchNormalization()(a_cnn)

a_rnn = L.LSTM(256,recurrent_dropout=0.2,return_sequences=True,unroll=True)(a_cnn)
# a_rnn = L.LSTM(256,recurrent_dropout=0.2,return_sequences=True,unroll=True)(a_rnn)
# a_rnn = L.LSTM(256,recurrent_dropout=0.2,return_sequences=True,unroll=True)(a_rnn)

a_avg = L.Lambda(lambda y: K.mean(y, axis=1))(a_rnn)
# r_avg = L.Lambda(lambda y: K.squeeze(y, axis=1))(r_avg)

f_inp = L.Concatenate(axis=1)([q_avg,r_avg,a_avg])

f_out = L.Dense(1024,activation='relu')(f_inp)
f_out = L.Dropout(0.3)(f_out)
f_out = L.Dense(1024,activation='relu')(f_out)
f_out = L.Dropout(0.3)(f_out)
f_out = L.Dense(1024,activation='relu')(f_out)
f_out = L.Dropout(0.3)(f_out)
f_out = L.Dense(1,activation='relu')(f_out)

model = M.Model(inputs=[q_inp,r_inp,a_inp],outputs=[f_out],name='Binary Classify')
optim = O.Adam(lr=0.001,decay=0.9)
model.compile(optimizer=optim,loss='mse',metrics=['mae'])

In [0]:
q_inp = L.Input(shape=(max_ql,v_len))
r_inp = L.Input(shape=(max_rl,v_len))
a_inp = L.Input(shape=(max_al,v_len))

inps = L.Concatenate(axis=1)([q_inp,r_inp,a_inp])
inps = L.Lambda(lambda y: K.mean(inps,axis=1))(inps)

d = L.Dense(1024, activation='relu')(inps)
d = L.Dense(1024, activation='relu')(d)
d = L.Dense(512, activation='relu')(d)
f_out = L.Dense(1, activation='relu')(d)

model = M.Model(inputs=[q_inp,r_inp,a_inp],outputs=[f_out],name='Binary Classify')
optim = O.Adam(lr=0.001,decay=0.9)
model.compile(optimizer=optim,loss='mse',metrics=['mae'])

In [0]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 17, 7)        0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           (None, 37, 7)        0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           (None, 193, 7)       0                                            
__________________________________________________________________________________________________
concatenate_4 (Concatenate)     (None, 247, 7)       0           input_10[0][0]                   
                                                                 input_11[0][0]                   
          

Model, features, data.. Run for all datasets and then decide which one to modify

In [0]:
g_un = []
for i in grades.keys():
  for j in grades[i].keys():
    if(grades[i][j] not in g_un):
      g_un.append(grades[i][j])

In [0]:
len(g_un)

10

In [0]:
split = int(0.6*len(qsv.keys()))

In [0]:
def train_test_split(qsv, rav, av, grades, split):
  train_qsv, test_qsv, train_rav, test_rav, train_av, test_av, train_grade, test_grade = [],[],[],[],[],[],[],[]
  for i in range(split):
    rep = len(av[i])
    for k in range(rep):
      train_qsv.append(qsv[i])
      train_rav.append(rav[i])
    for j in av[i].keys():
      train_av.append(av[i][j])
      train_grade.append(grades[i][j])
      
  for i in range(split,len(qsv)):
    rep = len(av[i].keys())
    for k in range(rep):
      test_qsv.append(qsv[i])
      test_rav.append(rav[i])
    for j in av[i].keys():
      test_av.append(av[i][j])
      test_grade.append(grades[i][j])

  return np.array(train_qsv), np.array(train_rav), np.array(train_av), np.array(train_grade), np.array(test_qsv), np.array(test_rav), np.array(test_av), np.array(test_grade)

In [0]:
train_qsv, train_rav, train_av, train_grade,test_qsv, test_rav, test_av, test_grade = train_test_split(qsv, rav, av, grades, split)

In [0]:
train_qsv.shape, train_rav.shape, train_av.shape, train_grade.shape

((324, 17, 7), (324, 37, 7), (324, 193, 7), (324,))

In [0]:
test_qsv.shape, test_rav.shape, test_av.shape, test_grade.shape

((247, 17, 7), (247, 37, 7), (247, 193, 7), (247,))

In [0]:
model.fit(x = [train_qsv, train_rav, train_av], y = [train_grade], validation_data=([test_qsv, test_rav, test_av],[test_grade]), batch_size=128, epochs=500)

Train on 324 samples, validate on 247 samples
Epoch 1/500
324/324 [==============================] - 1s 2ms/step - loss: 2.1157 - mean_absolute_error: 1.2375 - val_loss: 1.8777 - val_mean_absolute_error: 1.1449
Epoch 2/500
324/324 [==============================] - 0s 113us/step - loss: 2.1154 - mean_absolute_error: 1.2374 - val_loss: 1.8773 - val_mean_absolute_error: 1.1447
Epoch 3/500
324/324 [==============================] - 0s 106us/step - loss: 2.1150 - mean_absolute_error: 1.2373 - val_loss: 1.8769 - val_mean_absolute_error: 1.1446
Epoch 4/500
324/324 [==============================] - 0s 106us/step - loss: 2.1148 - mean_absolute_error: 1.2372 - val_loss: 1.8765 - val_mean_absolute_error: 1.1444
Epoch 5/500
324/324 [==============================] - 0s 144us/step - loss: 2.1144 - mean_absolute_error: 1.2371 - val_loss: 1.8761 - val_mean_absolute_error: 1.1443
Epoch 6/500
324/324 [==============================] - 0s 107us/step - loss: 2.1142 - mean_absolute_error: 1.2370 - val_l

In [0]:
model.evaluate(x=[test_qsv, test_rav, test_av], y=[test_grade], batch_size=128)

247/247 [==============================] - 0s 65us/step


[1.7401215938421397, 1.1031093650501267]

In [0]:
pred = model.predict(x=[test_qsv, test_rav, test_av])

In [0]:
pred1 = np.round(pred*2) / 2

In [0]:
p = pd.DataFrame(np.column_stack((pred1,test_grade)),columns=['Pred','Truth'])

In [0]:
p

,Pred,Truth
0,5.5,4
1,10.0,2.5
2,5.5,5
3,5.0,3
4,5.5,5
5,5.0,4.5
6,5.0,4.5
7,5.5,4.5
8,5.5,4
9,5.0,4.5
